## Skin Lesion Classification using VGG16

In [ ]:
import os
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import sklearn

In [ ]:
import numpy as np
import tensorflow as tf
import cv2
import os
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input

# Base path to your project folder in Google Drive
base_path = "/content/drive/MyDrive/btech_project"
base_path2= "/content/drive/MyDrive/btech_project/HAM_Dataset/NV"

# Construct full paths
model_filename = "vgg16_fe_phase1.keras"
image_filename = "ISIC_0024327.jpg"

model_path = os.path.join(base_path, model_filename)
img_path = os.path.join(base_path2, image_filename)

print(f"Loading model from: {model_path}")
# compile=False allows us to ignore custom metrics/loss functions
model = load_model(model_path, compile=False)


def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    # Create a sub-model that outputs the last conv layer + predictions
    grad_model = tf.keras.models.Model(
        inputs=[model.inputs],
        outputs=[model.get_layer(last_conv_layer_name).output, model.output]
    )

    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    # Calculate gradients
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # Global Average Pooling on gradients
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # Multiply each channel by importance
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # Apply ReLU and normalize
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

print(f"Processing image: {img_path}")
target_size = (224, 224)

# Load and Preprocess
try:
    img = image.load_img(img_path, target_size=target_size)
except FileNotFoundError:
    print(f"\n❌ ERROR: Image not found at {img_path}")
    print("Please check if the file name is correct and your Drive is mounted.")
    raise

img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = preprocess_input(img_array) # VGG16 specific scaling

# VGG16's last conv layer name
last_conv_layer_name = "block5_conv3"

# Temporarily remove softmax for better gradients
original_activation = model.layers[-1].activation
model.layers[-1].activation = None

# Generate Heatmap
print("Generating Grad-CAM heatmap...")
try:
    heatmap = make_gradcam_heatmap(img_array, model, last_conv_layer_name)
except ValueError as e:
    print(f"\n❌ ERROR: Layer '{last_conv_layer_name}' not found.")
    print("Run 'model.summary()' to find the name of the last 4D Conv layer.")
    raise e

# Restore softmax
model.layers[-1].activation = original_activation

# Load original image for display (using OpenCV)
img_cv2 = cv2.imread(img_path)
img_cv2 = cv2.resize(img_cv2, target_size)
img_cv2 = cv2.cvtColor(img_cv2, cv2.COLOR_BGR2RGB)

# Resize heatmap to match image size
heatmap_uint8 = np.uint8(255 * heatmap)

# Colorize heatmap (JET colormap)
jet = cm.get_cmap("jet")
jet_colors = jet(np.arange(256))[:, :3]
jet_heatmap = jet_colors[heatmap_uint8]

# Create RGB heatmap image
jet_heatmap = image.array_to_img(jet_heatmap)
jet_heatmap = jet_heatmap.resize(target_size)
jet_heatmap = image.img_to_array(jet_heatmap)

# Superimpose (Mix 60% original + 40% heatmap)
superimposed_img = jet_heatmap * 0.4 + img_cv2
superimposed_img = image.array_to_img(superimposed_img)

# Get Prediction for Title
preds = model.predict(img_array)
pred_idx = np.argmax(preds[0])
class_names = ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']
predicted_label = class_names[pred_idx]
confidence = preds[0][pred_idx]

# Plot
plt.figure(figsize=(14, 5))

plt.subplot(1, 3, 1)
plt.imshow(img_cv2)
plt.title("Original Image")
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(heatmap, cmap='viridis')
plt.title("Grad-CAM Heatmap")
plt.axis('off')

plt.subplot(1, 3, 3)
plt.imshow(superimposed_img)
plt.title(f"Overlay\nPred: {predicted_label} ({confidence:.2%})")
plt.axis('off')

plt.show()

### Creating Dataset

In [ ]:

import os

# Your base path in Drive
base_path = "/content/drive/MyDrive/btech_project"

# Folder where you want to store organized dataset
dataset_base = os.path.join(base_path, "HAM_Dataset")

# Create main folder if not exists
os.makedirs(dataset_base, exist_ok=True)

# Class names
classes = ["MEL", "NV", "BCC", "AKIEC", "BKL", "DF", "VASC"]

# Create subfolders
for cls in classes:
    class_path = os.path.join(dataset_base, cls)
    os.makedirs(class_path, exist_ok=True)

print("All folders created successfully!")


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:

!pip install opencv-python

In [ ]:

import cv2
import matplotlib.pyplot as plt
import os

images_dir_1 = "/content/drive/MyDrive/btech_project/HAM10000_images_part_1"
images_dir_2 = "/content/drive/MyDrive/btech_project/HAM10000_images_part_2"

# Combine image lists
all_images = (
    [os.path.join(images_dir_1, img) for img in os.listdir(images_dir_1)]
    +
    [os.path.join(images_dir_2, img) for img in os.listdir(images_dir_2)]
)

# Pick first image from combined list
img_path = all_images[0]
skin = cv2.imread(img_path)

plt.imshow(cv2.cvtColor(skin, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.show()

print("Image shape:", skin.shape)


In [ ]:

import pandas as pd
import os

base_path = "/content/drive/MyDrive/btech_project"
metadata_path = f"{base_path}/HAM10000_metadata.csv"

# Read metadata
df_labels = pd.read_csv(metadata_path)

# Map dx → folder label
dx_to_label = {
    "mel": "MEL",
    "nv": "NV",
    "bcc": "BCC",
    "akiec": "AKIEC",
    "bkl": "BKL",
    "df": "DF",
    "vasc": "VASC"
}

df_labels["label"] = df_labels["dx"].map(dx_to_label)

# Set image_id as index so we can do df_labels.loc[image_id, "label"]
df_labels.set_index("image_id", inplace=True)

df_labels.head()


In [ ]:

import numpy as np
from sklearn.utils import class_weight

classes = np.array(['AKIEC', 'BCC', 'BKL', 'DF', 'MEL', 'NV', 'VASC'])

class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=classes,
    y=df_labels["label"].values
)

class_weights



In [ ]:
class_wt_dict=dict(enumerate(class_weights))
class_wt_dict


In [ ]:

import os
import shutil
from tqdm import tqdm

base_path = "/content/drive/MyDrive/btech_project"

images_dir_1 = os.path.join(base_path, "HAM10000_images_part_1")
images_dir_2 = os.path.join(base_path, "HAM10000_images_part_2")

dataset_dir = os.path.join(base_path, "HAM_Dataset")

# Combine image file lists from both folders
all_images = (
    [os.path.join(images_dir_1, img) for img in os.listdir(images_dir_1)]
    +
    [os.path.join(images_dir_2, img) for img in os.listdir(images_dir_2)]
)

print("Total entries found:", len(all_images))

skipped_not_jpg = 0
skipped_no_label = 0

for img_path in tqdm(all_images):
    image = os.path.basename(img_path)  # e.g., ISIC_003412.jpg or .ipynb_checkpoints

    # 1) Skip anything that is not a .jpg file
    if not image.lower().endswith(".jpg"):
        skipped_not_jpg += 1
        continue

    # 2) Remove extension → get image_id
    fname = os.path.splitext(image)[0]   # ISIC_003412

    # 3) Skip if this image_id is not in df_labels index
    if fname not in df_labels.index:
        skipped_no_label += 1
        # Optional: print once or log
        # print("No label for:", fname)
        continue

    label = df_labels.loc[fname, "label"]    # MEL / NV / ...

    dst = os.path.join(dataset_dir, label, image)
    shutil.copyfile(img_path, dst)

print("Skipped non-JPG entries:", skipped_not_jpg)
print("Skipped files with no label:", skipped_no_label)


### Train Test Split

In [ ]:

!cp -r "/content/drive/MyDrive/btech_project/HAM_Dataset" "/content/HAM_Dataset"


In [ ]:
import os
bad_file = "/content/HAM_Dataset/NV/ISIC_0029218.jpg"

if os.path.exists(bad_file):
    os.remove(bad_file)
    print(f"✅ Deleted corrupted file: {bad_file}")
else:
    print("File already deleted.")

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import preprocess_input as vgg_preprocess


base_path = "/content/drive/MyDrive/btech_project"
# dataset_dir = os.path.join(base_path, "HAM_Dataset")
dataset_dir = "/content/HAM_Dataset"


df = df_labels.copy().reset_index()   # brings image_id out as a column
df.rename(columns={df.columns[0]: "image_id"}, inplace=True)  # just in case

df["filepath"] = df.apply(
    lambda r: os.path.join(dataset_dir, r["label"], r["image_id"] + ".jpg"),
    axis=1
)
train_df = train_test_split(
    df,
    test_size=0.2,
    stratify=df["label"],
    random_state=42
)

val_df, test_df = train_test_split(
    temp_df,
    test_size=0.5,
    stratify=temp_df["label"],
    random_state=42
)

# -------------------------------------------------------
# 3. GENERATORS
# -------------------------------------------------------
batch_size = 16
target_size = (224, 224)

train_datagen = ImageDataGenerator(
    preprocessing_function=vgg_preprocess,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_val_datagen = ImageDataGenerator(
    preprocessing_function=vgg_preprocess
)

train_gen = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col="filepath",
    y_col="label",
    target_size=target_size,
    color_mode='rgb',
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True
)

val_gen = test_val_datagen.flow_from_dataframe(
    dataframe=val_df,
    x_col="filepath",
    y_col="label",
    target_size=target_size,
    color_mode='rgb',
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=False
)

test_gen = test_val_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col="filepath",
    y_col="label",
    target_size=target_size,
    color_mode='rgb',
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=False
)

In [ ]:
# The ID of the corrupted image
bad_id = "ISIC_0029218"

# 1. Remove from standard splits (good practice to keep base clean)
train_df = train_df[train_df['image_id'] != bad_id]
val_df = val_df[val_df['image_id'] != bad_id]
test_df = test_df[test_df['image_id'] != bad_id]

# 3. REBUILD THE GENERATOR
# We re-point this to 'train_df_leaked' to ensure the corrupted file is gone
train_gen = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col="filepath",
    y_col="label",
    target_size=target_size,
    color_mode='rgb',
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True
)

In [ ]:
print("Train samples:", train_gen.samples)
print("Batch size:", train_gen.batch_size)
print("Steps per epoch:", train_gen.samples // train_gen.batch_size)

import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))

import time

start = time.time()
batch_x, batch_y = next(train_gen)
print("Time to load + preprocess one batch:", time.time() - start, "seconds")



##     

## Transfer Learning using VGG16 Model

## Model Training

In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input as vgg_preprocess
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

base_path = "/content/drive/MyDrive/btech_project"

# 1. Base model
image_size = 224
num_classes = 7

base_model = VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=(image_size, image_size, 3)
)

# 2. Add head (similar to MobileNet pattern)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

# 3. Freeze all layers first
for layer in model.layers:
    layer.trainable = False

# 4. Unfreeze last 3 conv layers (plus head which is automatically trainable)
conv_layers = [layer for layer in base_model.layers if "conv" in layer.name]
last_3_conv = conv_layers[-3:]

print("Unfreezing these conv layers:")
for layer in last_3_conv:
    layer.trainable = True
    print(layer.name)

# 5. Custom top-k metrics (like in MobileNet)
from tensorflow.keras.metrics import top_k_categorical_accuracy

def top_2_accuracy(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=2)

def top_3_accuracy(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=3)

# 6. Compile (smaller LR than MobileNet, VGG is heavier)
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss="categorical_crossentropy",
    metrics=["categorical_accuracy", top_2_accuracy, top_3_accuracy]
)

class_weights = {
    0: 1.0,  # AKIEC
    1: 1.0,  # BCC
    2: 1.0,  # BKL
    3: 1.0,  # DF
    4: 1.7,  # MEL (example)
    5: 1.0,  # NV
    6: 1.0,  # VASC
}

checkpoint = ModelCheckpoint(
    filepath=os.path.join(base_path, "vgg16_fe_phase1.keras"),
    monitor='val_categorical_accuracy',
    verbose=1,
    save_best_only=True,
    mode='max'
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_categorical_accuracy',
    factor=0.5,
    patience=2,
    verbose=1,
    mode='max',
    min_lr=1e-5
)

callbacks_list = [checkpoint, reduce_lr]

# 9. Train with generators
epochs = 20

history = model.fit(
    train_gen,                        # from your flow_from_dataframe
    epochs=epochs,
    validation_data=val_gen,
    class_weight=class_weights,
    callbacks=callbacks_list,
    verbose=1
)

# 10. Evaluate on validation set
val_loss, val_cat_acc, val_top_2_acc, val_top_3_acc = model.evaluate(val_gen)
print("val_loss:", val_loss)
print("val_cat_acc:", val_cat_acc)
print("val_top_2_acc:", val_top_2_acc)
print("val_top_3_acc:", val_top_3_acc)


In [ ]:
from sklearn.metrics import classification_report
import numpy as np
import tensorflow as tf
import os

# ---------------------------------------------------------
# 1. Save final model (full .keras model)
# ---------------------------------------------------------
save_path = os.path.join(base_path, "vgg16_fe_phase1.keras")
model.save(save_path)
print("Model saved at:", save_path)
import matplotlib.pyplot as plt

# 1. Extract the data from your history object
acc = history.history['categorical_accuracy']
val_acc = history.history['val_categorical_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

# 2. Define the X-axis (epochs)
# Since you are starting from scratch, we just count from 1 to the total number of epochs run
epochs_range = range(1, len(acc) + 1)

# 3. Plot Accuracy
plt.figure(figsize=(14, 5))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.grid(True)

# 4. Plot Loss
plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(loc='upper right')
plt.grid(True)

plt.show()

# ---------------------------------------------------------
# 2. Evaluate on the test set
# ---------------------------------------------------------
test_loss, test_cat_acc, test_top_2_acc, test_top_3_acc = model.evaluate(test_gen)

print("\n📌 TEST METRICS")
print("Test loss:", test_loss)
print("Test categorical accuracy:", test_cat_acc)
print("Test top-2 accuracy:", test_top_2_acc)
print("Test top-3 accuracy:", test_top_3_acc)

# ---------------------------------------------------------
# 3. Predict on the test generator
# ---------------------------------------------------------
pred_probs = model.predict(test_gen)
y_pred = np.argmax(pred_probs, axis=1)

# Extract true labels (Keras stores them here)
y_true = test_gen.classes     # integer labels from generator

# ---------------------------------------------------------
# 4. Map indices to class names
# ---------------------------------------------------------
idx_to_class = {v: k for k, v in test_gen.class_indices.items()}
class_names = [idx_to_class[i] for i in range(len(idx_to_class))]

print("\nClass indices mapping:", test_gen.class_indices)
print("Class names in order:", class_names)

# ---------------------------------------------------------
# 5. Classification report
# ---------------------------------------------------------
print("\n📌 CLASSIFICATION REPORT (HAM10000)\n")
print(classification_report(
    y_true,
    y_pred,
    target_names=class_names,
    digits=2
))


In [ ]:
print("\n📌 CONFUSION MATRIX\n")
from sklearn.metrics import classification_report, confusion_matrix # Added confusion_matrix

# Compute the matrix
cm = confusion_matrix(y_true, y_pred)

# Plotting the Heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names,
            yticklabels=class_names)
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

In [ ]:
# history = tl_model.fit(train_image_gen,
#                     epochs=20,
#                     validation_data = test_image_gen,
#                     class_weight=class_wt_dict,
#                     callbacks=callback_list)
base_path = "/content/drive/MyDrive/btech_project"
import os
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

# --------------------------
# 1. LOAD BASE VGG16
# --------------------------

vgg16_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# --------------------------
# 2. FREEZE ALL LAYERS FIRST
# --------------------------
for layer in vgg16_model.layers:
    layer.trainable = False

# --------------------------
# 3. UNFREEZE BLOCK 5 ONLY
# --------------------------
# for layer in vgg16_model.layers:
#     if "block5" in layer.name:
#         layer.trainable = True

# print("Unfrozen layers:")
# for layer in vgg16_model.layers:
#     if layer.trainable:
#         print(layer.name)

# --------------------------
# 4. ADD CLASSIFIER HEAD
# --------------------------
x = vgg16_model.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(7, activation='softmax')(x)

model = Model(inputs=vgg16_model.input, outputs=output)

# --------------------------
# COMPILE FOR PHASE 1
# --------------------------
model.compile(
    optimizer=Adam(learning_rate=1e-3),     # <-- higher LR
    loss="categorical_crossentropy",
    metrics=[
        "accuracy",
        tf.keras.metrics.TopKCategoricalAccuracy(k=2, name="top_2_acc"),
        tf.keras.metrics.TopKCategoricalAccuracy(k=3, name="top_3_acc")
    ]
)

checkpoint_phase1 = ModelCheckpoint(
    filepath=os.path.join(base_path, "phase1_best_weights.weights.h5"),
    save_best_only=True,
    save_weights_only=True,
    monitor='val_accuracy',
    verbose=1,
)

# --------------------------
# TRAIN PHASE 1
# --------------------------
history_phase1 = model.fit(
    train_gen,
    epochs=5,        # warm-up for 3–5 epochs
    validation_data=val_gen,
    class_weight=class_wt_dict,
    callbacks=[checkpoint_phase1],
    verbose=1
)

# --------------------------
# 5. COMPILE MODEL
# --------------------------
# model.compile(
#     loss="categorical_crossentropy",
#     optimizer=Adam(learning_rate=1e-4),    # small LR since we're fine-tuning
#     metrics=[
#     "accuracy",
#     tf.keras.metrics.TopKCategoricalAccuracy(k=2, name="top_2_acc"),
#     tf.keras.metrics.TopKCategoricalAccuracy(k=3, name="top_3_acc")
#     ]
# )

# model.summary()

# # --------------------------
# # 6. CALLBACKS
# # --------------------------
# base_path = "/content/drive/MyDrive/btech_project"

# lr_reduce = ReduceLROnPlateau(
#     monitor='val_accuracy',
#     factor=0.6,
#     patience=2,
#     mode='max',
#     min_lr=1e-5,
#     verbose=1
# )

# early_stop = EarlyStopping(
#     monitor="val_loss",
#     patience=3,
#     verbose=1,
#     restore_best_weights=True
# )

# checkpoint = ModelCheckpoint(
#     filepath=os.path.join(base_path, "best_model_vgg16.hdf5"),
#     save_best_only=True,
#     monitor='val_accuracy',
#     verbose=1
# )

# callbacks = [lr_reduce, early_stop, checkpoint]

# # --------------------------
# # 7. FIT MODEL
# # --------------------------
# history = model.fit(
#     train_gen,
#     epochs=30,                     # YOUR REQUIREMENT
#     validation_data=val_gen,
#     class_weight=class_wt_dict,   # from earlier
#     callbacks=callbacks
# )

# --------------------------
# 8. SAVE FINAL MODEL
# --------------------------
# save_path = os.path.join(base_path, "vgg16_fe.keras")
# model.save(save_path)

# print("Model saved at:", save_path)
# val_loss, val_cat_acc, val_top_2_acc, val_top_3_acc = model.evaluate(val_gen)

# print("val_loss:", val_loss)
# print("val_cat_acc:", val_cat_acc)
# print("val_top_2_acc:", val_top_2_acc)
# print("val_top_3_acc:", val_top_3_acc)



### Model Evaluation

In [ ]:
# # df=pd.DataFrame(tl_model.history.history)
# # df.to_csv('hist2.csv')
# # --------------------------
# # UNFREEZE ONLY BLOCK 5
# # --------------------------
# model.load_weights(os.path.join(base_path, "phase1_best_weights.weights.h5"))

# for layer in vgg16_model.layers:
#     layer.trainable = False
#     if "block5" in layer.name:
#         layer.trainable = True

# print("Trainable layers now:")
# for layer in vgg16_model.layers:
#     if layer.trainable:
#         print(layer.name)

# # --------------------------
# # COMPILE FOR PHASE 2
# # --------------------------
# model.compile(
#     optimizer=Adam(learning_rate=1e-4),     # <-- smaller LR for fine tuning
#     loss="categorical_crossentropy",
#     metrics=[
#         "accuracy",
#         tf.keras.metrics.TopKCategoricalAccuracy(k=2, name="top_2_acc"),
#         tf.keras.metrics.TopKCategoricalAccuracy(k=3, name="top_3_acc")
#     ]
# )

# # Callbacks
# lr_reduce = ReduceLROnPlateau(monitor='val_accuracy', factor=0.6, patience=2, mode='max', min_lr=1e-5, verbose=1)
# early_stop = EarlyStopping(monitor="val_loss", patience=3, verbose=1, restore_best_weights=True)
# checkpoint = ModelCheckpoint(
#     os.path.join(base_path, "best_model_vgg16.weights.h5"),
#     save_best_only=True,
#     save_weights_only=True,
#     monitor='val_accuracy',
#     verbose=1
# )

# callbacks = [lr_reduce, early_stop, checkpoint]

# # --------------------------
# # TRAIN PHASE 2
# # --------------------------
# history_phase2 = model.fit(
#     train_gen,
#     epochs=20,                # <-- full fine-tuning
#     validation_data=val_gen,
#     class_weight=class_wt_dict,
#     callbacks=callbacks,
#     verbose=1
# )

# # --------------------------
# # SAVE FINAL MODEL
# # --------------------------
# val_loss, val_cat_acc, val_top_2_acc, val_top_3_acc = model.evaluate(val_gen)

# print("Final Validation Loss:", val_loss)
# print("Final Validation Top-1 Accuracy:", val_cat_acc)
# print("Final Validation Top-2 Accuracy:", val_top_2_acc)
# print("Final Validation Top-3 Accuracy:", val_top_3_acc)
# model.save(os.path.join(base_path, "vgg16_fe.keras"))
# print("Model saved to vgg16_fe.keras")


